In [ ]:
openai_key = "kk"

In [ ]:
%pip install weaviate-client

In [ ]:
import weaviate

client = weaviate.connect_to_embedded(
    headers = {
        "X-OpenAI-Api-Key": openai_key,
    },
    version = "latest"
)

In [ ]:
client.collections.delete_all()

In [ ]:
from weaviate import classes as wvc 

In [ ]:
collection = client.collections.create(
    name="Bebidas",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(model="text-embedding-3-small"),
    generative_config=wvc.config.Configure.Generative.openai(
        max_tokens=1000,
        temperature=0.7
    )
)

In [ ]:
client.collections.list_all()

In [ ]:
coco = client.collections.get("Bebidas")
coco.config.get().to_dict() # weaviate já cria uma coleção por padrão

In [ ]:
coco.data.insert({"text": "Água Natual. Diretamente da Bahia"})

In [ ]:
query = coco.query.fetch_objects(include_vector=True) # olha meu vetor aq foi gerado

In [ ]:
query = coco.query.bm25("Gelado")
for o in query.objects:
    print("propriedades: ", o.properties)


In [ ]:
query = coco.query.near_text("É um chá do Paraguay",
    return_metadata=wvc.query.MetadataQuery(distance=True),
    single_prompt="Gere por favor um tweet pra essa bebida. use emojis e tenha um tom engraçado. Use {text}"
)
for o in query.objects:
    print("propriedades: ", o.properties, " distância: ", o.metadata.distance)

In [ ]:
query = coco.generate.near_text("É um chá do Paraguay",
    return_metadata=wvc.query.MetadataQuery(distance=True),
    single_prompt="Gere por favor um tweet pra essa bebida. use emojis e tenha um tom engraçado. Use o campo {text}"
)
for o in query.objects:
    print("propriedades: ", o.properties, " distância: ", o.metadata.distance, " Generated: ", o.generated)

In [ ]:
client.close()

In [ ]:
client.is_ready()

In [ ]:
client.get_meta()